In [6]:
import os
import cv2
import xml.etree.ElementTree as ET
import shutil
import random
import matplotlib.pyplot as plt

In [2]:
base_dir = 'images'
jpeg_dir = 'jpeg'
xml_dir = 'xml'

os.makedirs(jpeg_dir, exist_ok=True)
os.makedirs(xml_dir, exist_ok=True)

for filename in os.listdir(base_dir):
    if filename.endswith('.jpeg'):
        shutil.copy(os.path.join(base_dir, filename), os.path.join(jpeg_dir, filename))
    elif filename.endswith('.xml'):
        shutil.copy(os.path.join(base_dir, filename), os.path.join(xml_dir, filename))

print('Files copied separately in their respective directory.')

Files copied separately in their respective directory.


In [3]:
jpeg_count = len([filename for filename in os.listdir(jpeg_dir) if filename.endswith('.jpeg')])
xml_count = len([filename for filename in os.listdir(xml_dir) if filename.endswith('.xml')])

print(f"Jpeg count: {jpeg_count}")
print(f"xml count: {xml_count}")


Jpeg count: 228
xml count: 225


In [7]:
jpeg_dir = "jpeg"
xml_dir = "xml"
dataset_dir = "dataset"

train_images_dir = os.path.join(dataset_dir, 'images', 'train')
train_labels_dir = os.path.join(dataset_dir, 'labels', 'train')
val_images_dir = os.path.join(dataset_dir, 'images', 'val')
val_labels_dir = os.path.join(dataset_dir, 'labels', 'val')

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

def convert_voc_to_yolo(xml_file, img_width, img_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    yolo_data = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = 0

        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # Convert to YOLO format
        x_center = (xmin + xmax) / 2 / img_width
        y_center = (ymin + ymax) / 2 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_data

image_files = []

for filename in os.listdir(jpeg_dir):
    if filename.endswith('.jpeg'):
        image_files.append(filename)

random.shuffle(image_files)

split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

for filename in train_files + val_files:
    image_path = os.path.join(jpeg_dir, filename)

    if filename in train_files:
        target_image_dir = train_images_dir
        target_label_dir = train_labels_dir
    else:
        target_image_dir = val_images_dir
        target_label_dir = val_labels_dir

    shutil.copy(image_path, os.path.join(target_image_dir, filename))

    xml_file_path = os.path.join(xml_dir, os.path.splitext(filename)[0] + '.xml')

    if os.path.exists(xml_file_path):
        img = cv2.imread(image_path)
        img_height, img_width, _ = img.shape

        yolo_labels = convert_voc_to_yolo(xml_file_path, img_width, img_height)
        label_file_path = os.path.join(target_label_dir, f"{os.path.splitext(filename)[0]}.txt")

        with open(label_file_path, 'w') as f:
            for line in yolo_labels:
                f.write(line + '\n')
    
    else:
        print(f"No XML file found for {filename}, skipping label generation.")

print("Dataset structure created with train and validation sets.")


No XML file found for N182.jpeg, skipping label generation.
No XML file found for N249.jpeg, skipping label generation.
No XML file found for N180.jpeg, skipping label generation.
Dataset structure created with train and validation sets.


In [4]:
yaml_content = """train: C:/Users/2003s/Desktop/number-plate-detection/dataset/images/train
val: C:/Users/2003s/Desktop/number-plate-detection/dataset/images/train

nc: 1 # Number of classes
names: ['number_plate'] # List of class names
"""

with open('dataset/dataset.yaml', 'w') as file:
    file.write(yaml_content)

print("dataset.yaml file has been created.")

dataset.yaml file has been created.


In [3]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

# model.train(data='dataset/dataset.yaml', epochs=100, imgsz=640)
model.train(data='dataset/dataset.yaml', epochs=100, device='cuda')

Ultralytics YOLOv8.1.10 🚀 Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset/dataset.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

train: Scanning C:\Users\2003s\Desktop\number-plate-detection\dataset\labels\train.cache... 181 images, 1 backgrounds, 0 corrupt: 100%|██████████| 182/182 [00:00<?, ?it/s]
val: Scanning C:\Users\2003s\Desktop\number-plate-detection\dataset\labels\train.cache... 181 images, 1 backgrounds, 0 corrupt: 100%|██████████| 182/182 [00:00<?, ?it/s]


Plotting labels to runs\detect\train5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.18G        nan        nan        nan         12        640: 100%|██████████| 12/12 [00:12<00:00,  1.07s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.83it/s]

                   all        182        181          0          0          0          0


...


100 epochs completed in 0.459 hours.
Optimizer stripped from runs\detect\train5\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train5\weights\best.pt, 6.3MB

Validating runs\detect\train5\weights\best.pt...
Ultralytics YOLOv8.1.10 🚀 Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.83it/s]

                   all        182        181          0          0          0          0


Speed: 0.8ms preprocess, 14.9ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\train5


AttributeError: 'DetMetrics' object has no attribute 'curves_results'. See valid attributes below.

    This class is a utility class for computing detection metrics such as precision, recall, and mean average precision
    (mAP) of an object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (tuple of str): A tuple of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass

In [13]:
import cv2
import matplotlib.pyplot as plt
model = YOLO('runs/detect/train2/weights/best.pt')

test_images_dir = 'TEST'
output_dir = 'test_results'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(test_images_dir):
    if filename.endswith(('.jpeg', '.jpg', '.png')):
        image_path = os.path.join(test_images_dir, filename)

        results = model.predict(source=image_path, save=True, save_dir=output_dir)

        if results:
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            for result in results:
                boxes = result.boxes
                if len(boxes) == 0:
                    print(f"No detections for {filename}.")
                    continue

                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf[0].cpu().item()
                    class_id = int(box.cls[0].cpu().item())

                    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 2)
                    label = f'Class {class_id} {confidence:.2f}'
                    cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            plt.imshow(img)
            plt.title(f'Detections for {filename}')
            plt.axis('off')
            plt.show()

        else:
            print(f"No results for {filename}")

print("DONE!")


image 1/1 c:\Users\2003s\Desktop\number-plate-detection\TEST\TEST.jpeg: 544x640 1 number_plate, 104.0ms
Speed: 33.5ms preprocess, 104.0ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)
Results saved to runs\detect\predict5


<Figure size 640x480 with 1 Axes>

DONE!


In [14]:
models_dir = 'models'

os.makedirs(models_dir, exist_ok=True)

model.save('runs/detect/train2/weights/best.pt')

print("Model weights have been saved.")

Model weights have been saved.
